# PDF Extraction

1. First run the rename function below
2. Then extract the yml files.

In [6]:
%load_ext autoreload
%autoreload 2
import sys
import os
import re
import pandas as pd
import shutil
from pathlib import Path
#from dateutil.parser import parse
#from fuzzyparsers import parse_date
#import timelib

sys.path.append('../..')
import data.dataframe_preparation as preparation

purge_existing_folder = False
extract_text = True
reports_input_dir = "/Volumes/Mac_Backup/Data/fin-disclosures-nlp/stoxx600/unprocessed"
master_output_file = os.path.join(reports_input_dir, "Firm_AnnualReport_TrainingV4.csv")
random_seed = 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Script for renaming files to AR_YYYY.pdf format
Handles cases such as 2018-2019 (will take the higher year), and 123232019 (will ignore).

In [2]:
company_paths = preparation.get_company_paths(reports_input_dir)
for company_dir in company_paths:
    company_files = preparation.get_reports_paths(company_dir.path)

    for p in company_files:
        filename = str(p.stem)

        if purge_existing_folder:
            try:
                shutil.rmtree(os.path.join(company_dir.path, filename))
            except:
                print("No file found for: ", filename)
        
        potential_years = re.findall(r"(?<!\d)((?:199|200|201|202)[0-9]{1})(?!\d)", filename)
        if len(potential_years) != 1:
            if len(potential_years) and abs(int(potential_years[0]) - int(potential_years[1])) == 1:
                year = max(potential_years)
                print(filename, ": found a range of years, taking the higher one...")
            
            else:
                print("Ambigious years found for ", filename)
                print(potential_years)
                print("=========")
                break
        else:
            year = potential_years[0]
        
        if year:
            os.rename(p, Path(p.parent, f"AR_{year}.pdf"))
    
    cleaned_company_name = re.sub(r'\W+', '', str(company_dir.name))
    if cleaned_company_name != company_dir.name:
        new_folder = os.path.join(os.path.dirname(company_dir), cleaned_company_name)
        os.rename(company_dir, new_folder)

### Extract text from PDF

In [7]:
if extract_text:
    ! python ../../data/pipeline.py "$reports_input_dir"

Processing file /Volumes/Mac_Backup/Data/fin-disclosures-nlp/stoxx600/unprocessed/IndutradeAB/AR_2007.pdf into /Volumes/Mac_Backup/Data/fin-disclosures-nlp/stoxx600/unprocessed/IndutradeAB/AR_2007 with parser tika
Processing file /Volumes/Mac_Backup/Data/fin-disclosures-nlp/stoxx600/unprocessed/IndutradeAB/AR_2010.pdf into /Volumes/Mac_Backup/Data/fin-disclosures-nlp/stoxx600/unprocessed/IndutradeAB/AR_2010 with parser tika
Processing file /Volumes/Mac_Backup/Data/fin-disclosures-nlp/stoxx600/unprocessed/IndutradeAB/AR_2011.pdf into /Volumes/Mac_Backup/Data/fin-disclosures-nlp/stoxx600/unprocessed/IndutradeAB/AR_2011 with parser tika
Processing file /Volumes/Mac_Backup/Data/fin-disclosures-nlp/stoxx600/unprocessed/IndutradeAB/AR_2012.pdf into /Volumes/Mac_Backup/Data/fin-disclosures-nlp/stoxx600/unprocessed/IndutradeAB/AR_2012 with parser tika
No outline for PDF found!
Processing file /Volumes/Mac_Backup/Data/fin-disclosures-nlp/stoxx600/unprocessed/IndutradeAB/AR_2013.pdf into /Volume

In [6]:
master_df = preparation.get_df(input_path=reports_input_dir, include_text=False, include_page_no=False, include_toc=False)
master_df['should_label'] = True
master_df['is_labelled'] = False
master_df = master_df.sample(frac=1, random_state=1)

In [7]:

master_df.to_csv(master_output_file)
